In [1]:
import os
import torch
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import hashlib


gamma = 1/2
g = 10

secret_key = "123"

seed = 10000

dataset='covertype'


original_file = '../dataset/covtype_with_key.subset.data'
origin = pd.read_csv(original_file)

np.random.seed(seed)

cover_types = origin['Cover_Type'].unique()
cover_types.sort()

def hash_mod(key, mod_value):
    combined = f"{secret_key}{key}"
    hash_value = int(hashlib.sha256(combined.encode()).hexdigest(), 16)  
    return hash_value % mod_value  

columns_of_interest = ['Elevation', 'Aspect'] 
origin[columns_of_interest] = origin[columns_of_interest].fillna(0)  

def first_two_digits(x):
    if x == 0:
        return "00"
    digits = str(x).lstrip('0.').replace('.', '') 
    if len(digits) == 1: 
        return digits + "0" 
    return digits[:2] 


np.random.seed(seed)
candidate_set = cover_types
shuffled_cover_types = list(cover_types)
np.random.shuffle(shuffled_cover_types)
    
for idx in range(len(origin)):
    selected_data = origin.loc[idx, columns_of_interest]
    first_two_digits_data = selected_data.apply(first_two_digits)
    composite_numbers = ''.join(first_two_digits_data.values)
    
    if(hash_mod(composite_numbers, g) != 2):
        continue
   
    half_size = len(shuffled_cover_types) // 2
    green_domain = shuffled_cover_types[:half_size]
    red_domain = shuffled_cover_types[half_size:]
    if origin.loc[idx, 'Cover_Type'] in red_domain:
        perturb_value = np.random.choice(green_domain)
        origin.loc[idx, 'Cover_Type'] = perturb_value    


results = {
    'watermarked_data': origin
}

np.save(f"{dataset}-{seed}.npy", results)






In [3]:
# detect
import os
import torch
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import hashlib


gamma = 1/2 
g = 10

seed = 10000
dataset='covertype'
secret_key = "123"


def hash_mod(key, mod_value):
    combined = f"{secret_key}{key}"
    hash_value = int(hashlib.sha256(combined.encode()).hexdigest(), 16) 
    return hash_value % mod_value  

def first_two_digits(x):
    if x == 0:
        return "00"
    digits = str(x).lstrip('0.').replace('.', '') 
    if len(digits) == 1: 
        return digits + "0"  
    return digits[:2]  

columns_of_interest = ['Elevation', 'Aspect']  


loaded_results = np.load(f"{dataset}-{seed}.npy", allow_pickle=True).item()
# watermarked_data = loaded_results['watermarked_data']
watermarked_data = pd.read_csv("../dataset/covtype_with_key.subset.data")

watermarked_data[columns_of_interest] = watermarked_data[columns_of_interest].fillna(0)  # 填充NA为0

cover_types = watermarked_data['Cover_Type'].unique()
cover_types.sort()  

green_cell = 0
n_cell = 0
np.random.seed(seed)
shuffled_cover_types = list(cover_types)
np.random.shuffle(shuffled_cover_types)

for idx in range(len(watermarked_data)):

    selected_data = watermarked_data.loc[idx, columns_of_interest]
    first_two_digits_data = selected_data.apply(first_two_digits)
    composite_numbers = ''.join(first_two_digits_data.values)
    
    if(hash_mod(composite_numbers, g) != 2):
        continue
    
    n_cell += 1
    
    candidate_set = cover_types

    

    half_size = len(shuffled_cover_types) // 2
    green_domain = shuffled_cover_types[:half_size]
    red_domain = shuffled_cover_types[half_size:]

    if watermarked_data.loc[idx, 'Cover_Type'] in green_domain:
        green_cell += 1

z_score = (green_cell - n_cell/2) / math.sqrt(n_cell/4)

print("The average z-score is ",z_score)







The average z-score is  -5.674975293786874
